<a href="https://colab.research.google.com/github/MukundIsGod/SceneDetection/blob/master/inceptionv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-2.4.1
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
ERROR: Could not find a version that satisfies the requirement tensorflow-2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-2.4.1
     |████████████████████████████████| 349.7MB 52kB/s 
     |████████████████████████████████| 501kB 40.2MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 3.1MB 31.9MB/s 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#data_root='/content/drive/My Drive/scenes'
data_root='/content/drive/My Drive/MIT/indoorCVPR_09/Images'

In [ ]:
from tensorflow.keras.applications.inception_v3  import InceptionV3

In [ ]:
pre_model = InceptionV3(input_shape = (229, 229, 3), include_top = False, weights = 'imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in pre_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

x = layers.Flatten()(pre_model.output)
x = layers.Dense(1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense (67, activation = 'softmax')(x)
model = Model(pre_model.input, x)
model.compile(optimizer=Adam(lr=0.001), loss = 'categorical_crossentropy', metrics=['acc'])

In [ ]:
IMAGE_SHAPE = (229, 229)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="validation", 
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE)


/content/drive/My Drive/MIT/indoorCVPR_09/Images
Found 3098 images belonging to 67 classes.
Found 12519 images belonging to 67 classes.


In [ ]:
inc_history = model.fit_generator(train_generator, validation_data = valid_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 2456s 25s/step - loss: 15.4139 - acc: 0.0362 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 2/10
100/100 [==============================] - 1087s 11s/step - loss: 15.5741 - acc: 0.0338 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 3/10
100/100 [==============================] - 1160s 12s/step - loss: 15.3675 - acc: 0.0469 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 4/10
100/100 [==============================] - 1119s 11s/step - loss: 15.5086 - acc: 0.0378 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 5/10
100/100 [==============================] - 64s 641ms/step - loss: 15.4784 - acc: 0.0397 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 6/10
100/100 [==============================] - 66s 656ms/step - loss: 15.4229 - acc: 0.0435 - val_loss: 15.4938 - val_acc: 0.0387
Epoch 7/10
100/100 [==============================] - 65s 646ms/step - loss: 15.5389 - acc: 0.0359 - val_loss: 15.4914 - val_acc: 0.0387
Epoch 8/10
100/100 [=====================

In [ ]:
!mkdir "saved_models"
saved_model_path = "./saved_models/mask_unmask.h5" 
model.save(saved_model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

In [ ]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
tf_model_predictions = model.predict(val_image_batch)
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels
tf_pred_dataframe.head(10)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("/content/drive/My Drive/inception.tflite")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()
